Данная домашняя работа составлена на основании источников указанных в ноутуке урока и [домашнего задания](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_notebooks/homeworks/hw05_logit_rf_credit_scoring.ipynb) сообщества Open Data Science

In [ ]:
# отключим предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import random 
import numpy as np

### Задание 1
_(вспоминаем теорему Кондорсе «о жюри присяжных».)_

На конкурсе 5 членов жюри, каждый член жюри по отдельности с вероятностью 70% может угадать следующего победителя конкурса. С какой вероятностью все вместе члены жюри могут угадать следующего победителя, если решение принимается большинством голосов?

Варианты ответов:
- **73.32%**
- **77.44%**
- **83.7%**
- **87.4%**
- **90.12%**

In [ ]:
# ваш код здесь

### Задание 2
Используя подход будстрап выборок предсказать среднее значение сгенерированного массива.
При каком размере подвыборки получается наиболее точное значение среднего?

Варианты ответов:
- **1**
- **5**
- **10**
- **15**

In [ ]:
seed=17
random.seed(seed) # зафиксировать рандом для воспроизводимости
np.random.seed(seed)

dataset = np.array([random.uniform(0, 333) for _ in range(100)])
print("Истинное средне значение сгенерированных данных: {}".format(np.mean(dataset)))

In [ ]:
# Код по предсказанию среднего
def get_bootstrap_samples(data, n_samples, seed=0):
    ''' функция для генерации подвыборок с помощью бутстрэпа '''
    np.random.seed(seed)
    # ваш код
    pass

random.seed(seed) # зафиксировать рандом для воспроизводимости
for sub_size in []:
    # ваш код
    pass

Для выполнения следующих заданий вам понадобится следующий датасет:

https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/credit_scoring_sample.csv

Данные представлены следующим образом:

Прогнозируемая переменная

    SeriousDlqin2yrs ----Человек не выплатил данный кредит в течение 90 дней; возможные значения 1/0

Независимые признаки

   - age ---- Возраст заёмщика кредитных средств; тип - integer
   - NumberOfTime30-59DaysPastDueNotWorse ----Количество раз, когда человек имел просрочку выплаты других кредитов более 30-59 дней, но не больше в течение последних двух лет; тип - integer
   - DebtRatio ---- Ежемесячный отчисления на задолжености(кредиты,алименты и т.д.) / совокупный месячный доход percentage; тип - real
   - MonthlyIncome ----Месячный доход в долларах; тип - real
   - NumberOfTimes90DaysLate ----Количество раз, когда человек имел просрочку выплаты других кредитов более 90 дней; тип - integer
   - NumberOfTime60-89DaysPastDueNotWorse---- Количество раз, когда человек имел просрочку выплаты других кредитов более 60-89 дней, но не больше в течение последних двух лет; ; тип - integer
   - NumberOfDependents ----Число человек в семье кредитозаёмщика; тип - integer


In [ ]:
## Сделаем функцию, которая будет заменять NaN значения на медиану в каждом столбце таблицы 
def delete_nan(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table

## Считываем данные
data = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse_open/master/data/credit_scoring_sample.csv', sep =';')

## Выберем названия всех признаков из таблицы, кроме прогнозируемого
independent_columns_names = data.columns.values
independent_columns_names = [x for x in data if x != 'SeriousDlqin2yrs']

## Применяем функцию, заменяющую все NaN значения на медианное значение соответствующего столбца
table =delete_nan(data)

## Разделяем таргет и признаки 
X =table[independent_columns_names]
y = table['SeriousDlqin2yrs']

### Задание 3 
**Простая реализация случайного леса**

Реализуйте свой собственный случайный лес с помощью `DecisionTreeClassifier` с параметрами поумолчанию. В нашем лесу будет 10 деревьев, предсказанные вероятности которых вам нужно усреднить. 

Краткая спецификация: 
 - Используйте основу ниже
 - В методе `fit` в цикле (`i` от 0 до `n_estimators-1`) фиксируйте seed.
 - Зафиксировав seed, выберите **без замещения** `max_features` признаков, сохраните список выбранных id признаков в `self.feat_ids_by_tree`
 - Также сделайте bootstrap-выборку (т.е. **с замещением**) из множества id объектов 
 - Метод `fit` возвращает текущий экземпляр класса `RandomForestClassifierCustom`, то есть `self`
 - В методе `predict_proba` опять нужен цикл по всем деревьям. У тестовой выборки нужно взять те признаки, на которых соответсвующее дерево обучалось, и сделать прогноз вероятностей (`predict_proba` уже для дерева). Метод должен вернуть усреднение прогнозов по всем деревьям.

Проведите кросс-валидацию. Какое получилось среднее значение ROC AUC на кросс-валидации? Выберите самое близкое значение.

Варианты ответа:
- **0.813**
- **0.823**
- **0.833**
- **0.843**
- **0.853**

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold,cross_val_score

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=10, max_depth=7, max_features=6, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        
        self.trees = []
        self.feat_ids_by_tree = []
        
    def fit(self, X, y):
        ## ваш код здесь
        pass
    
    def predict_proba(self, X):
        ## ваш код здесь
        pass

Зафиксируем кросс-валидацию: стратифицированная, 5 разбиений с перемешиванием, не забываем про `random_state`.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [ ]:
rf = RandomForestClassifierCustom().fit(X.values, y.values)

In [ ]:
cv_aucs = cross_val_score(RandomForestClassifierCustom(max_depth=7, max_features=6),\
                          X.values, y.values, scoring="roc_auc", cv=skf)
print("Средняя ROC AUC для собственного случайного леса:", np.mean(cv_aucs))

### Задание 4
Тут сравним нашу собственную реализацию случайного леса с sklearn-овской. Для этого воспользуйтесь RandomForestClassifier(n_jobs=1, random_state=seed), укажите все те же значения max_depth и max_features, что и раньше. Какое среднее значение ROC AUC на кросс-валидации мы в итоге получили? Выберите самое близкое значение.

Варианты ответа:
- **0.823**
- **0.833**
- **0.843**
- **0.853**


In [ ]:
from sklearn.ensemble import RandomForestClassifier
## ваш код здесь

### Задание 5
Подберите с помощью GridSearchCV (RandomForestClassifier(random_state=17)) глубину деревьев (max_depth) и количество параметров (max_features).
Чему равны их значения?

In [ ]:
max_depth_values = # ваш код
max_features_values = # ваш код
forest_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}